# Create SPECFEM3D Project
1. Unpickle events and staions
1. Read and map moment tensors to events
1. Prep observed station data for writing to ascii files
1. Setup and test DATA directories and specfem3d file names
1. Create directories and write CMTSOLUTION, STATION, STATION_ADJOINT, and trace ascii files in separate dictories for each event
1. Test P-arrival pick on one trace
1. Plot p-arrivals for all traces of a chosen event
1. Create p-arriavls (trigger) dictionary for all events
1. Write adjoint source window parameter files

#### 1. Unpickle events and stations 

In [ ]:
import pickle
from gnam.events.gevents import gevents as gevents
from gnam.events.gstations import gstations as gstations
from gnam.events.munge.knmi import correct_station_depths as csd_f

data_root_dir = '../../../data_notebooks'

#Unpickle events
print('Unpickling Events')
f = open('../../../data_notebooks/pickled/events.pickle', 'rb')
dill_events = pickle.load(f)
f.close()
    

#Unpickle stations
print('Unpickling Station Traces')
f = open('../../../data_notebooks/pickled/straces.pickle', 'rb')
dill_straces = pickle.load(f)
f.close()

# This is kind of hokey, but it works for now.
# Some of the stations depths do not follow the 
# 50, 100, 150, 200 meter depths -- possibly because
# the boreholes are slanted. To correct for this,
# a hard coded "patch/update" is applied. See the
# code for details and update values.
dill_straces.correct_stations(csd_f)

print('Done')

#### 2. Read and map moment tensors to events

In [ ]:
from gnam.events.mtensors import mtensors
import pandas as pd
import numpy as np

#Get model bounding box needed for adjusting strike
gf_bbox = dill_events.getBBox()

#Read moment tensors
gf_mts = mtensors('../../../data_notebooks/data/event_moments.csv',gf_bbox)
    
# get event catalog of the events withing the bounding box
e_cat = dill_events.getIncCatalog()

# This is a bit hokey, but it works. Here we update the
# event time from the moment tensor CSV file with thouse
# from the event catalog
gf_mts.update_utcdatetime(e_cat)

# Create a dictionary that maps moment tensors to events
e2mt_dict = gf_mts.map_events_2_tensors(e_cat)
e2mt_keys = e2mt_dict.keys()

# Print a comparison of events to moment tensors
for key in e2mt_keys:
    print('UTC: event[%d][Date] = %s' %(key,e_cat[key].origins[0].time))
    print('UTC:    MT[%d][Date] = %s' %(key,e2mt_dict[key]['Date']))
    print('Mag: event[%d][Date] = %s' %(key,e_cat[key].magnitudes[0].mag))
    print('Mag:    MT[%d][Date] = %s' %(key,e2mt_dict[key]['ML']))
    print()

#### 3. Create Streams and dictionars of all event station catalogs that are included

In [ ]:
from obspy import Stream

#create streams and map eventid to stream
e_stream_dict = {}
ce_stream_dict = {}
for ekey in e2mt_keys:
    '''
    e_st  = Stream()
    e_st += dill_straces.getStreamZ(ekey,3) 
    e_st += dill_straces.getStreamZ(ekey,4) 
    e_st.resample(1000)
    e_stream_dict[ekey] = e_st
    '''
    
    #get boreholes 3 and 4 streams for the event
    e1_st3 = dill_straces.getStream1(ekey,3).copy()
    e1_st4 = dill_straces.getStream1(ekey,4).copy()
    e2_st3 = dill_straces.getStream2(ekey,3).copy()
    e2_st4 = dill_straces.getStream2(ekey,4).copy()
    ez_st3 = dill_straces.getStreamZ(ekey,3).copy()
    ez_st4 = dill_straces.getStreamZ(ekey,4).copy()

    #combine component streams
    e_st3 = e1_st3 + e2_st3 + ez_st3
    e_st4 = e1_st4 + e2_st4 + ez_st4

    #reorder streams so that each station component is contiguous
    te_st3 = Stream()
    te_st4 = Stream()

    ns = len(ez_st3)
    for i in range(ns):
        te_st3 += e_st3[i+2*ns]
        te_st3 += e_st3[i+ns]
        te_st3 += e_st3[i]

        te_st4 += e_st4[i+2*ns]
        te_st4 += e_st4[i+ns]
        te_st4 += e_st4[i]
    
    e_st3 = te_st3
    e_st4 = te_st4
    
    #rotate the streams to ZNE components (obspy and the FDSN info is used)
    inv3 = dill_straces.get_inventory(3)
    inv4 = dill_straces.get_inventory(4)
    
    trZ = e_st3[0]
    trZ_name = trZ.stats.network + '.' + trZ.stats.station + '..' + trZ.stats.channel
    #trZ_orient = {}
    trZ_orient = inv3.get_orientation(trZ_name)
    print('%s orientaion' %(trZ_name), trZ_orient)
    tr1 = e_st3[1]
    tr1_name = tr1.stats.network + '.' + tr1.stats.station + '..' + tr1.stats.channel
    #tr1_orient = {}
    tr1_orient = inv3.get_orientation(tr1_name)
    print('%s orientaion' %(tr1_name), tr1_orient)
    tr2 = e_st3[2]
    tr2_name = tr2.stats.network + '.' + tr2.stats.station + '..' + tr2.stats.channel
    #tr2_orient = {}
    tr2_orient = inv3.get_orientation(tr2_name)
    print('%s orientaion' %(tr2_name), tr2_orient)
    
    e_st3.rotate(method="->ZNE", inventory=inv3)
    e_st4.rotate(method="->ZNE", inventory=inv4)

    #resample and map event to stream
    e_st = e_st3 + e_st4
    ce_st = e_st.copy() #keep a pre-resampled set for P-arrival picking (less memory)
    #e_st.resample(1000) #better to resample while iterating. 
    e_stream_dict[ekey] = e_st
    ce_stream_dict[ekey] = ce_st

In [ ]:
print(e_stream_dict[3].sort()[0:6])

#### 6. Test P-arrival pick on one trace

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from obspy.core import read
from obspy.signal.trigger import ar_pick


czne_st = ce_st.copy()

df = czne_st[0].stats.sampling_rate
f1 = 3.0
f2 = 5.0
lta_p = 1.0
sta_p = 0.1
lta_s = 4.0
sta_s = 1.0
m_p = 2
m_s = 8
l_p = 0.1
l_s = 0.2

p_sfudge = 0.75
s_efudge = 0.75

czne_st[0:3].filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)

p_pick, s_pick = ar_pick(czne_st[0].data, czne_st[1].data, czne_st[2].data, df,
                         f1, f2, lta_p, sta_p, lta_s, sta_s, m_p, m_s, l_p, l_s)

#print(p_pick)
#print(s_pick)

fig, ax = plt.subplots(1,figsize=(14,5))
#print('czne_st[0]:\n',vars(czne_st[0]))
z_data = czne_st[0].data
n_data = czne_st[1].data
e_data = czne_st[2].data
dt     = czne_st[0].stats.delta
nt     = czne_st[0].count()
t      = czne_st[0].times()
#print('dt:',dt)
#print('nt:',nt)
#print('df:',df)
ax.plot(t, z_data,c='black')
ax.plot(t, n_data,c='green')
ax.plot(t, e_data,c='orange')
amin_z = np.min(z_data)
amax_z = np.max(z_data)
amin_n = np.min(n_data)
amax_n = np.max(n_data)
amin_e = np.min(e_data)
amax_e = np.max(e_data)
amin = np.min([amin_z,amin_n,amin_e])*2
amax = np.max([amax_z,amax_n,amax_e])*2
#print('amax:',amax)
#print('amin:',amin)
ax.vlines(x=p_pick, ymin=amin, ymax=amax, colors='red',linestyle='dashed')
ax.vlines(x=p_pick-p_sfudge, ymin=amin, ymax=amax, colors='red')
ax.vlines(x=s_pick, ymin=amin, ymax=amax, colors='blue',linestyle='dashed')
ax.vlines(x=s_pick+s_efudge, ymin=amin, ymax=amax, colors='blue')

plt.grid(b=True, which='major', color='#666666', linestyle='-')
plt.minorticks_on()
plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)

plt.show()

#### 7. Plot p-arrivals for all traces of a chosen event

In [ ]:
#choose event between min and max ekey (some events don't have inverted moment tensors yet)
import numpy.random as rand
#rand.seed(2)
cekeys = ce_stream_dict.keys()
min_cekey = min(cekeys)
max_cekey = max(cekeys)
print('min key:',min_cekey)
print('max key:',max_cekey)
rand_cekey = rand.randint(min_cekey, max_cekey)
#rand_cekey = min_cekey
rand_cekey = 3
print(rand_cekey)

In [ ]:
def get_triggers(p_0,s_0,p_trig_pad,s_trig_pad):
    if s_0 < p_0:
        s_0 = 2.1*p_0
    if (s_0 - p_0) < 2.0:
        s_0 = p_0 + 2.0
        
    #if the padded p-pick is less then zero, set to zero
    p_trig = p_0 - p_trig_pad
    if p_trig < 0.0:
        p_trig = 0.0
    #if the padded s-pick is greater than the last time sample, set to the last sample
    s_trig = s_0 + s_trig_pad
    if t[-1] <= s_trig:
        s_trig = t[-1]
        
    return (p_trig,p_0,s_0,s_trig)

In [ ]:
#plot the picks on the trances for the chosen event.
import obspy
from obspy import Stream
from obspy import Trace
from obspy import UTCDateTime
import matplotlib.pyplot as plt
import numpy as np
import copy 
from obspy.signal.trigger import recursive_sta_lta
from obspy.signal.trigger import plot_trigger

dc_real = copy.deepcopy(ce_stream_dict[rand_cekey])
df = ce_st[0].stats.sampling_rate
f1 = 4.0
f2 = 16.0
lta_p = 1.5
sta_p = 0.5
lta_s = 2
sta_s = 0.5
m_p = 8
m_s = 8
l_p = 0.5
l_s = 0.5

p_trig_pad = 0.75
s_trig_pad = 0.75

dc_real.filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)

plt_h = 2
plt_w = 14
ntrace = len(dc_real)//3
plt_scale = int(1*len(dc_real[:3*ntrace:3]))
fig, ax = plt.subplots(plt_scale,figsize=(plt_w,plt_h*plt_scale))
fig.tight_layout()
zshft = int(0)
nshft = int(1)
eshft = int(2)
shft = zshft
triggers = np.zeros((ntrace,4))
trig_dict = {}
for i in range(len(dc_real[:3*ntrace:3])):
    
    #dc_real[shft+i*3].filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)
    
    #real_data = dc_real[shft+i*3].data
    
    dt     = dc_real[i*3].stats.delta
    nt     = dc_real[i*3].count()
    t      = dc_real[i*3].times()
    
    p_0, s_0 = ar_pick(dc_real[i*3].data, dc_real[i*3+1].data, dc_real[i*3+2].data, df, 
                       f1, f2, lta_p, sta_p, lta_s, sta_s, m_p, m_s, l_p, l_s)
    
    
    '''
    if s_0 < p_0:
        s_0 = 2.1*p_0
    if (s_0 - p_0) < 2.0:
        s_0 = p_0 + 2.0
    assert 2.0 <= (s_0 - p_0)
        
    #if the padded p-pick is less then zero, set to zero
    p_trig = p_0 - p_trig_pad
    if p_trig < 0.0:
        p_trig = 0.0
    #if the padded s-pick is greater than the last time sample, set to the last sample
    s_trig = s_0 + s_trig_pad
    if t[-1] <= s_trig:
        s_trig = t[-1]
    '''
    p_trig, p_0, s_0, s_trig = get_triggers(p_0,s_0,p_trig_pad,s_trig_pad)
        
    triggers[i,0] = p_trig
    triggers[i,1] = p_0
    triggers[i,2] = s_0
    triggers[i,3] = s_trig
    trig_dict[dc_real[shft+i*3].stats.station] = np.array(triggers[i,:])
    print('event %d: trig %s:' %(rand_cekey,dc_real[shft+i*3].stats.station), np.array(triggers[i,:]))
    
    ax[i].plot(t, dc_real[i*3].data,c='black',zorder=2)
    ax[i].plot(t, dc_real[i*3+1],c='green',zorder=1)
    ax[i].plot(t, dc_real[i*3+2],c='orange',zorder=0)
    
    amin = np.min(dc_real[i*3].data)
    amin = np.min(np.array([amin,np.min(dc_real[i*3+1].data)]))
    amin = np.min(np.array([amin,np.min(dc_real[i*3+2].data)]))
    amax = np.max(dc_real[i*3].data)
    amax = np.max(np.array([amax,np.max(dc_real[i*3+1].data)]))
    amax = np.max(np.array([amax,np.max(dc_real[i*3+2].data)]))
    ax[i].vlines(x=p_0, ymin=amin, ymax=amax, colors='red',linestyle='dashed')
    ax[i].vlines(x=p_trig, ymin=amin, ymax=amax, colors='red')
    ax[i].vlines(x=s_0, ymin=amin, ymax=amax, colors='blue',linestyle='dashed')
    ax[i].vlines(x=s_trig, ymin=amin, ymax=amax, colors='blue')
    
    rstation = dc_real[i*3].stats.station
    rstation += '.' + dc_real[shft+i*3].stats.channel
    
    overlay_title = 'Station:' + str(rstation)
    ax[i].set_title(overlay_title)
    
    ax[i].grid(b=True, which='major', color='#666666', linestyle='-')
    ax[i].minorticks_on()
    ax[i].grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
plt.show()


#### 8. Create p-arriavls (trigger) dictionary for all events

In [ ]:
event_list = [3,4,5]
event_trigger_dict = {}
for eid in event_list:

    #make copy so that it can be filtered to a good band for picking
    dc_real = ce_stream_dict[eid].copy()
    
    dc_real.filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)

    ntrace = len(dc_real)//3
    triggers = np.zeros((ntrace,4))
    trig_dict = {}
    for i in range(len(dc_real[:3*ntrace:3])):

        t      = dc_real[i*3].times()

        p_0, s_0 = ar_pick(dc_real[i*3].data, dc_real[i*3+1].data, dc_real[i*3+2].data, df, 
                           f1, f2, lta_p, sta_p, lta_s, sta_s, m_p, m_s, l_p, l_s)


        p_trig, p_0, s_0, s_trig = get_triggers(p_0,s_0,p_trig_pad,s_trig_pad)
        
        triggers[i,0] = p_trig
        triggers[i,1] = p_0
        triggers[i,2] = s_0
        triggers[i,3] = s_trig
        trig_dict[dc_real[i*3].stats.station] = np.array(triggers[i,:])
        print('event %d: trig %s:' %(eid,dc_real[shft+i*3].stats.station), np.array(triggers[i,:]))
        
    #end sloop over station (and component) traces
        
    event_trigger_dict[eid] = trig_dict
    
#end loop over events
print(event_trigger_dict[4])

#### Read SPECFEM syn data and store into Streams

In [ ]:
import obspy
from obspy import Stream
from obspy import Trace
from obspy import UTCDateTime
import fnmatch
import os
import numpy as np
import pandas as pd


dpath = '../../../data_notebooks/data/syn/'
subdir = 'SYN/'

emax = 5
eshift = 2 # run dirs start at 0001, but not all obs events have a Moment Tensor
esyn_dict = {}
# read all filenames for each event
for ekey in e2mt_keys:
    esyn_key = ekey - eshift
    if emax < ekey or esyn_key <= 0:
        continue
    rdir = 'run%s/' %(str(esyn_key).zfill(4))
    fqdp = dpath + rdir + subdir
    print('fqdp:',fqdp)
    
    obs_event = dill_events.getIncCatalog()[ekey]
    etime = obs_event.origins[0].time
    print('etime:',etime)
    
    syn_st = Stream()
    for file in os.listdir(fqdp):
        if fnmatch.fnmatch(file, '*semd'):
            fqn = fqdp + file
            df = pd.io.parsers.read_csv(fqn,sep="\s+",header=None, usecols=[0,1])
            data = df[[1]].to_numpy().astype(np.float32).flatten()
            fhdr = file.split('.')
            # Fill header attributes
            stats = {'network': fhdr[0], 'station': fhdr[1], 'location': '',
                     'channel': fhdr[2], 'npts': len(data), 'delta': 0.001}
            
            syntime = df[[0]].to_numpy().astype(np.float64).flatten()
            print('starttime:        ',etime)
            stats['starttime'] = etime + syntime[0]
            print('starttime shifted:',stats['starttime'])
            st = Stream([Trace(data=data, header=stats)])
            print('Stream Before Resample:\n', st)
            print('endtime shifted:  ',st[0].stats.endtime)
            print('sampling:',st[0].stats.sampling_rate)
            st[0].resample(200)
            print('resampling:',st[0].stats.sampling_rate)
            print('Stream After Resample:\n', st)
            print('delta:  ',st[0].stats.delta)
            print('resamp starttime:  ',st[0].stats.starttime)
            print('resamp endtime:    ',st[0].stats.endtime)
            print('resamp len(data)   ',len(st[0].data))
            endtime = st[0].stats.endtime
            st[0] = st[0].slice(etime,endtime)
            print('sliced starttime:  ',st[0].stats.starttime)
            print('sliced endtime:    ',st[0].stats.endtime)
            print('sliced len(data)   ',len(st[0].data))
            syn_st += st
            
    esyn_dict[ekey] = syn_st.sort()



In [ ]:
print(esyn_dict[3][0:6])

#### Register Syn and Obs Data

In [ ]:
#First change station names for ease of comparison
rot_ce_stream_dict = {}
for key in ce_stream_dict:
    ce_st = ce_stream_dict[key].copy()
    for itr in range(len(ce_st)//3):
        dataE = None
        dataN = None
        itrE = None
        itrN = None
        #print('iitr:',3*itr)
        for i in range(3):
            iitr = 3*itr + i
            tr = ce_st[iitr]
            comp_char = tr.stats.channel[2] 
            if comp_char == 'Z':
                continue
            #print('Before - station.channel:', tr.stats.station + '.' + tr.stats.channel)
            if comp_char == 'E':
                tr.stats.channel = 'HHX'
                itrE = iitr
                dataE = tr.data.copy()
            elif comp_char == 'N':
                tr.stats.channel = 'HHY'
                itrN = iitr
                dataN = tr.data.copy()
            #print('After  - station.channel:', tr.stats.station + '.' + tr.stats.channel)
            #print()
        
        dataNp = dataN*np.cos(-2*np.pi/3) + dataE*np.sin(-2*np.pi/3)
        dataEp = dataE*np.cos(-np.pi/6) + dataN*np.sin(-np.pi/6)
        #print('itrN:',itrN)
        #print('itrE:',itrE)
        ce_st[itrN].data[:] = dataNp[:]
        ce_st[itrE].data[:] = dataEp[:]
        
    rot_ce_stream_dict[key] = ce_st

#Now register the data
reg_obs_st_dict = {}
reg_syn_st_dict = {}
for key in esyn_dict:
    print('KEY:',key)
    obs_st = rot_ce_stream_dict[key].copy()
    obs_st.sort()
    syn_st = esyn_dict[key].copy()
    for i in range(len(obs_st)):
        print('syn.station,real.station = (%s,%s)' %(syn_st[i].stats.station,obs_st[i].stats.station))
        print('syn.starttime  = %s' %(syn_st[i].stats.starttime))
        print('real.starttime = %s' %(obs_st[i].stats.starttime))
        print('syn.endtime    = %s' %(syn_st[i].stats.endtime))
        print('real.endtime   = %s' %(obs_st[i].stats.endtime))
        print('syn.delta      = %s' %(syn_st[i].stats.delta))
        print('real.delta     = %s' %(obs_st[i].stats.delta))
        print('syn.len        = %s' %(len(syn_st[i].data)))
        print('real.len       = %s' %(len(obs_st[i].data)))
        start_t = obs_st[i].stats.starttime
        end_t   = syn_st[i].stats.endtime
        obs_st[i] = obs_st[i].slice(start_t,end_t)
        syn_st[i] = syn_st[i].slice(start_t,end_t)
        print('sliced syn.starttime   = %s' %(syn_st[i].stats.starttime))
        print('sliced real.starttime  = %s' %(obs_st[i].stats.starttime))
        print('sliced syn.endtime     = %s' %(syn_st[i].stats.endtime))
        print('sliced real.endtime    = %s' %(obs_st[i].stats.endtime))
        #FIXME: machine epsilon issue? Mitigation below
        syn_st[i].stats.starttime = obs_st[i].stats.starttime
        print('sliced syn.starttime   = %s' %(syn_st[i].stats.starttime))
        print('sliced syn.endtime     = %s' %(syn_st[i].stats.endtime))
        print('sliced syn.delta       = %s' %(syn_st[i].stats.delta))
        print('sliced real.delta      = %s' %(obs_st[i].stats.delta))
        print('sliced syn.len         = %s' %(len(syn_st[i].data)))
        print('sliced real.len        = %s' %(len(obs_st[i].data)))
        print('sliced syn.times()[0]  = %s' %(syn_st[i].times()[0]))
        print('sliced real.times()[0] = %s' %(obs_st[i].times()[0]))
        print('sliced syn.times()[-1]  = %s' %(syn_st[i].times()[-1]))
        print('sliced real.times()[-1] = %s' %(obs_st[i].times()[-1]))
        print()
        print()
    reg_obs_st_dict[key] = obs_st
    reg_syn_st_dict[key] = syn_st
    
 


### Define Signal to Noise -win to +win (s) of Trigger Function

In [ ]:
def calc_s2n_ratio_power_4tr(tr,triggers,win):
    
        
        p_0 = triggers[1]
        dt  = tr.stats.delta
        assert 0 < dt
        
        ip_0 = int(p_0/dt)
        nwin = int(win/dt)
        assert 0 < nwin
        i_rwin = ip_0 + nwin + 1
        i_lwin = ip_0 - nwin
        
        if i_lwin < 0:
            i_lwin = 0
        
        lsum = np.sum(tr.data[i_lwin:ip_0]**2)
        lsum /= nwin
        
        rsum = np.sum(tr.data[ip_0+1:i_rwin]**2)
        rsum /= nwin
        
        ratio = 1000000
        if lsum != 0:
            ratio = rsum/lsum
        
        return (ratio, tr.times()[i_lwin], tr.times()[i_rwin])
        
        

#### Define Plotting Function for Comparison of Obs vs Syn

In [ ]:
def plot_obs_v_syn(ekey,obs_st,syn_st,trig_dict,comp,f1,f2,plt_h,plt_w,win=0.5,fname=None):
    
    assert len(obs_st) == len(syn_st)
    
    assert comp == 'X' or comp == 'Y' or comp == 'Z'
    
    icomp = 0
    if comp == 'Y':
        icomp = 1
    if comp == 'Z':
        icomp = 2
    
    c_obs_st = obs_st.copy()
    c_syn_st = syn_st.copy()

    c_obs_st.filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)
    c_syn_st.filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)
    
    ntrace = len(c_obs_st)//3

    plt_scale = int(ntrace)
    fig, ax = plt.subplots(plt_scale,figsize=(plt_w,plt_h*plt_scale))
    fig.tight_layout()
    
    for i in range(ntrace):
        itr = 3*i + icomp
        
        triggers = trig_dict[c_obs_st[itr].stats.station]
        #print('event %d: trig %s:' %(ekey,c_obs_st[itr].stats.station), np.array(triggers[:]))
        
        p_trig = triggers[0]
        p_0    = triggers[1]
        s_0 = triggers[2]
        s_trig = triggers[3]
        
        s2n, lwt, rwt = calc_s2n_ratio_power_4tr(c_obs_st[itr],triggers,win)

        omin = np.min(c_obs_st[itr].data)
        omax = np.max(c_obs_st[itr].data)
        onorm = 1.0/np.max([np.abs(omin),np.abs(omax)])
        #onorm = 1.0
        o_pdata = np.zeros_like(c_obs_st[itr].data)
        o_pdata[:] = onorm*c_obs_st[itr].data[:]
        
        smin = np.min(c_syn_st[itr].data)
        smax = np.max(c_syn_st[itr].data)
        snorm = 1.0/np.max([np.abs(smin),np.abs(smax)])
        #snorm = 1.0
        s_pdata = np.zeros_like(c_syn_st[itr].data)
        s_pdata[:] = snorm*c_syn_st[itr].data[:]
        
        amin = np.min(np.array([np.min(o_pdata),np.min(s_pdata)]))
        amax = np.max(np.array([np.max(o_pdata),np.max(s_pdata)]))

        t = c_obs_st[itr].times()
        ax[i].plot(t, o_pdata,c='black',label='obs',zorder=1)
        ax[i].plot(t, s_pdata,c='green',label='syn',zorder=0)
        
        ax[i].vlines(x=p_0, ymin=amin, ymax=amax, colors='red',linestyle='dashed')
        ax[i].vlines(x=p_trig, ymin=amin, ymax=amax, colors='red')
        ax[i].vlines(x=s_0, ymin=amin, ymax=amax, colors='blue',linestyle='dashed')
        ax[i].vlines(x=s_trig, ymin=amin, ymax=amax, colors='blue')
        if win != None:
            ax[i].axvspan(lwt, p_0, label='pre P-arrival',color="red", alpha=0.2)
            ax[i].axvspan(p_0, rwt, label='post P-arrival',color="blue", alpha=0.2)

        station = c_obs_st[itr].stats.station
        station += '.' + c_obs_st[itr].stats.channel

        overlay_title = 'Event: %d,  Station.Channel-%s: @%d-%dHz,  SNR: %.2f' %(ekey,str(station),int(f1),int(f2),s2n)
        ax[i].set_title(overlay_title)
        ax[i].legend()

        ax[i].grid(b=True, which='major', color='#666666', linestyle='-')
        ax[i].minorticks_on()
        ax[i].grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
        
        
    if fname != None:
        assert win != None
        ofname = '%s_%s_%d-%dHz_s2n%.1f_w%.2f.png' %(fname,str(station),int(f1),int(f2),s2n,win)
        fig.savefig(ofname, bbox_inches='tight')
    plt.show()

### Plot Obs vs Syn for each component

In [ ]:
!pwd

In [ ]:
ekey = 3
obs_st = reg_obs_st_dict[ekey]
syn_st = reg_syn_st_dict[ekey]
trig_dict = event_trigger_dict[ekey]
f1 = 4.0
f2 = 16.0
plt_h = 2
plt_w = 14
xcomp = 'X'
ycomp = 'Y'
zcomp = 'Z'
#print(trig_dict)

#### Compare X-component

In [ ]:
plot_obs_v_syn(ekey,obs_st,syn_st,trig_dict,xcomp,f1,f2,plt_h,plt_w,win=1.0)

#### Compare Y-component

In [ ]:
plot_obs_v_syn(ekey,obs_st,syn_st,trig_dict,ycomp,f1,f2,plt_h,plt_w,win=1.0)

#### Compare Z-component

In [ ]:
#plot_obs_v_syn(ekey,obs_st,syn_st,trig_dict,zcomp,f1,f2,plt_h,plt_w,win=1.0,fname='Event-%d'%(ekey))
plot_obs_v_syn(ekey,obs_st,syn_st,trig_dict,zcomp,f1,f2,plt_h,plt_w,win=1.0)

### Define Function to create dictionary of SNRs to be use for removing stations

In [ ]:
c_obs_st = obs_st.copy()
print('Before:', c_obs_st[0:3])
f_obs_st = obs_st.copy()
x_st = f_obs_st.select(component='X')
y_st = f_obs_st.select(component='Y')
z_st = f_obs_st.select(component='Z')

ntrace = len(x_st)

for i in range(ntrace):
    
    x_tr = x_st[i]
    y_tr = y_st[i]
    z_tr = z_st[i]
    
    xstation = x_tr.stats.station
    ystation = y_tr.stats.station
    zstation = z_tr.stats.station
    
    assert xstation == ystation and xstation == zstation
    
    if xstation == 'G013':
        c_obs_st.remove(x_tr)
        c_obs_st.remove(y_tr)
        c_obs_st.remove(z_tr)
    
print('Before:', c_obs_st[0:3])



In [ ]:
def remove_traces_by_snr(ekey,obs_st,trig_dict,f1,f2,plt_h,plt_w,win=1.0,z_snr_thold=10.0,x_snr_thold=5.0,y_snr_thold=5.0):
    
    f_obs_st = obs_st.copy()
    k_obs_st = obs_st.copy()
    r_obs_st = Stream()
    
    x_st = obs_st.select(component='X')
    y_st = obs_st.select(component='Y')
    z_st = obs_st.select(component='Z')

    f_obs_st.filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)
    f_x_st = f_obs_st.select(component='X')
    f_y_st = f_obs_st.select(component='Y')
    f_z_st = f_obs_st.select(component='Z')
    
    assert len(f_x_st) == len(f_y_st) and len(f_x_st) == len(f_z_st)
    assert len(x_st) == len(y_st) and len(x_st) == len(z_st)
    assert len(f_x_st) == len(x_st)
    
    ntrace = len(f_x_st)
    
    keep_dict   = {}
    remove_dict = {}

    for i in range(ntrace):
            
        x_tr = x_st[i]
        y_tr = y_st[i]
        z_tr = z_st[i]

        xstation = x_tr.stats.station
        ystation = y_tr.stats.station
        zstation = z_tr.stats.station
        
        xchan = x_tr.stats.channel
        ychan = y_tr.stats.channel
        zchan = z_tr.stats.channel

        assert xstation == ystation and xstation == zstation

        triggers = trig_dict[xstation]

        p_trig = triggers[0]
        p_0    = triggers[1]
        s_0 = triggers[2]
        s_trig = triggers[3]

        x_s2n, lwt, rwt = calc_s2n_ratio_power_4tr(f_x_st[i],triggers,win)
        y_s2n, lwt, rwt = calc_s2n_ratio_power_4tr(f_y_st[i],triggers,win)
        z_s2n, lwt, rwt = calc_s2n_ratio_power_4tr(f_z_st[i],triggers,win)

        if z_s2n < z_snr_thold or x_s2n < x_snr_thold or y_s2n < y_snr_thold:
            print()
            print('Removing Station: %s' %(xstation))
            print('x_s2n:',x_s2n)
            print('y_s2n:',y_s2n)
            print('z_s2n:',z_s2n)
            remove_dict[xstation + '.' + xchan] = x_s2n
            remove_dict[ystation + '.' + ychan] = y_s2n
            remove_dict[zstation + '.' + zchan] = z_s2n
            k_obs_st.remove(x_tr)
            k_obs_st.remove(y_tr)
            k_obs_st.remove(z_tr)
            r_obs_st += x_st[i]
            r_obs_st += y_st[i]
            r_obs_st += z_st[i]
        else:
            keep_dict[xstation + '.' + xchan] = x_s2n
            keep_dict[ystation + '.' + ychan] = y_s2n
            keep_dict[zstation + '.' + zchan] = z_s2n
            
    return (k_obs_st.sort(),r_obs_st.sort(),keep_dict,remove_dict)

c_obs_st = obs_st.copy()
print('Before:', c_obs_st[0:3])
print('Before:', len(c_obs_st))
k_st,r_st,k_dict,r_dict = remove_traces_by_snr(ekey,c_obs_st,trig_dict,f1,f2,plt_h,plt_w,win=1.0)
print()
print('After Keep: ', k_st[0:3])
print('After Keep: ', len(k_st))
print()
print('After Remo: ', r_st[0:3])
print('After Remo: ', len(r_st))
print()
#print('Keep:\n',k_dict)
#print('Remo:\n',r_dict)

### Define Signal to Noise -0.75s to +0.75s of Trigger Function

In [ ]:
def calc_s2n_ratio_power(ekey,obs_st,trig_dict,comp,win,f1,f2):
    
    assert comp == 'X' or comp == 'Y' or comp == 'Z'
    
    s2n_dict = {}
    
    icomp = 0
    if comp == 'Y':
        icomp = 1
    if comp == 'Z':
        icomp = 2
    
    c_obs_st = obs_st.copy()

    c_obs_st.filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)
    
    ntrace = len(c_obs_st)//3

    for i in range(ntrace):
        itr = 3*i + icomp
        
        s_key = c_obs_st[itr].stats.station
        c_key = c_obs_st[itr].stats.channel
        tr_key = s_key + '.' + c_key
        triggers = trig_dict[s_key]
        #print('event %d: trig %s:' %(ekey,c_obs_st[itr].stats.station), np.array(triggers[:]))
        
        p_0    = triggers[1]
        dt = c_obs_st[itr].stats.delta
        
        ip_0 = int(p_0/dt)
        nwin = int(win/dt)
        i_rwin = ip_0 + nwin + 1
        i_lwin = ip_0 - nwin
        
        if i_lwin < 0:
            i_lwin = 0
        
        #print('left win len: ',len(c_obs_st[itr].data[i_lwin:ip_0]))
        lsum = np.sum(c_obs_st[itr].data[i_lwin:ip_0]**2)
        lsum /= nwin
        
        #print('right win len:',len(c_obs_st[itr].data[ip_0+1:i_rwin]))
        rsum = np.sum(c_obs_st[itr].data[ip_0+1:i_rwin]**2)
        rsum /= nwin
        
        s2n_dict[tr_key] = rsum/lsum
        
    return s2n_dict
        

In [ ]:
win = 1 #seconds
f1 = 2.0
f2 = 6.0
s2n = calc_s2n_ratio_power(ekey,obs_st,trig_dict,'Z',win,f1,f2)
print('s2n:\n',s2n)

## Finished